# Supplementary Figures - Proof-of-concept CROP-seq KO15 screen Euler diagrams

In [1]:
# set correct working directory -> project folder
getwd()
setwd('..')
getwd()

[1] "/home/sreichl/projects/bmdm-stim/workflow/notebooks"

[1] "/home/sreichl/projects/bmdm-stim"

In [2]:
# source the figure theme, libraries and utility functions
source("./src/figure_theme.R")

In [3]:
# for euler plots
library(eulerr)

# configs

In [4]:
# configs
data_path <- file.path('results','KO15')
results_path <- file.path('results','figures','supp_small_screen')

In [5]:
# make directories if not exist
dir.create(results_path, showWarnings = FALSE, recursive = TRUE)

# DEG Euler diagrams of selected mixscape DEA results

In [6]:
selected_kos <- c('mStat2','mIrf9','mEp300')

In [7]:
conditions <- c('untreated', 'LO28_2h', 'LO28_6h')

In [8]:
dea_results <- list()

for (cond in conditions){
    cond_dea <- data.frame()
    for (ko in selected_kos){
        tmp_dea <- read.csv(file.path(data_path, 'DEA_condition_mixscape', paste0('KO15_DEA_RNA_',cond,'_',ko,'.csv')))
        tmp_dea$ko <- ko
        colnames(tmp_dea)[1] <- 'gene'
        
        if(dim(cond_dea)[1]==0){
            cond_dea <- tmp_dea
        }else{
            cond_dea <- rbind(cond_dea, tmp_dea)
        }
    }
    # filter for stat. sign.
    cond_dea <- cond_dea[cond_dea$p_val_adj<0.05,]
    # add group information
    cond_dea$direction <- lapply(cond_dea$avg_log2FC, function(x) if(x>0)'up' else 'down')
    cond_dea$group <- paste0(cond_dea$ko,'_',cond_dea$direction)
    
    dea_results[[cond]] <- cond_dea
}

In [9]:
dea_sets <- list()

for (cond in conditions){
    cond_sets <- data.frame(matrix(ncol = 0, nrow = length(unique(dea_results[[cond]]$gene))))
    rownames(cond_sets) <- unique(dea_results[[cond]]$gene)
    for (group in unique(dea_results[[cond]]$group)){
        tmp_genes <- unique(dea_results[[cond]][grepl(group, dea_results[[cond]]$group, fixed = TRUE),'gene'])
        cond_sets[tmp_genes, group] <- TRUE
    }
    cond_sets[is.na(cond_sets)] <- FALSE
    
    dea_sets[[cond]] <- cond_sets
}

In [10]:
head(dea_sets[['untreated']])

,mStat2_down,mStat2_up,mIrf9_down,mIrf9_up,mEp300_down,mEp300_up
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Irf7,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE
Bst2,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE
Ifi27l2a,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE
Isg15,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE
Ly6e,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE
Ifi44,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE


In [22]:
# 'mStat2_down','mIrf9_down','mEp300_up'
for (cond in conditions){
    svg(filename=file.path(results_path, paste0("KO15_DEG_",cond,"_euler_",'mStat2down_','mIrf9down_','mEp300up',".svg")), width = 5, height = 5)

    dea_tmp <- dea_sets[[cond]][c('mStat2_down','mIrf9_down','mEp300_up')]
    colnames(dea_tmp) <- gsub(pattern = '^.', replacement = "", colnames(dea_tmp))
    colnames(dea_tmp) <- gsub(pattern = '_', replacement = " ", colnames(dea_tmp))
    
    dea_fit <- euler(dea_tmp, shape = "ellipse")
#     dea_fit <- euler(dea_sets[[cond]][c('mStat2_down','mIrf9_down','mEp300_up')], shape = "ellipse")
    print(plot(dea_fit, quantities = TRUE))

    dev.off()
}

In [23]:
# 'mStat2_up','mIrf9_up','mEp300_down'
for (cond in conditions){
    svg(filename=file.path(results_path, paste0("KO15_DEG_",cond,"_euler_",'mStat2up_','mIrf9up','mEp300down',".svg")), width = 5, height = 5)
    
    dea_tmp <- dea_sets[[cond]][c('mStat2_up','mIrf9_up','mEp300_down')]
    colnames(dea_tmp) <- gsub(pattern = '^.', replacement = "", colnames(dea_tmp))
    colnames(dea_tmp) <- gsub(pattern = '_', replacement = " ", colnames(dea_tmp))

    dea_fit <- euler(dea_tmp, shape = "ellipse")
#     dea_fit <- euler(dea_sets[[cond]][c('mStat2_up','mIrf9_up','mEp300_down')], shape = "ellipse")
    print(plot(dea_fit, quantities = TRUE))

    dev.off()
}

In [24]:
# save data frames
for (cond in conditions){
    write.csv(dea_sets[[cond]], file.path(results_path, paste0("KO15_DEG_",cond,"_euler_",'mStat2_','mIrf9_','mEp300',".csv")))
}